In [1]:
import re
import ast
from csv import reader
def load_inputs(path):
    with open('input_puzzles.csv', 'r') as f:
        csv_reader = reader(f)
        list_rows = list(csv_reader)
        for row in list_rows[1:]:
            initial = parse_input(row[0].strip())
            goal = parse_input(row[1].strip())
    return initial, goal

def parse_input(x):
    x = re.sub('\(', '[', x)
    x = re.sub('\)', ']', x)
    x = re.sub(';', ',', x)
    x = ast.literal_eval(x)
    return x
# path = 'inputs.txt'
path = 'input_puzzles.csv'
s0, g = load_inputs(path)

In [2]:
print(s0)
print(g)

[[6, 1, 2], [7, 8, 3], [5, 4, 9]]
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]
